In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pylab as pl
import numpy as np
%matplotlib inline

In [2]:
age_gender = pd.read_csv('data/age_gender_bkts.csv')
countries = pd.read_csv('data/countries.csv')
sessions = pd.read_csv('data/sessions.csv')
train_users = pd.read_csv('data/train_users_2.csv')

In [3]:
#cleaning up

train_users.replace("-unknown-", np.nan, inplace = True)
from datetime import datetime
new_timestamp = train_users.timestamp_first_active.astype(str)
dates = pd.to_datetime(pd.Series([datetime.strptime(date, '%Y%m%d%H%M%S') for date in new_timestamp]))

train_users['timestamp_first_active'] = dates
train_users['date_account_created'] = pd.to_datetime(train_users.date_account_created)
train_users['date_first_booking'] = pd.to_datetime(train_users.date_first_booking)
train_users.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,NaN,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [4]:
#this will make it a lot easier when merging and also to avoid confusion about what id can be about
train_users.rename(columns = {'id':'user_id'}, inplace = True)
df = train_users.merge(sessions, on = 'user_id', how = 'left')
df.head()

,user_id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,action,action_type,action_detail,device_type,secs_elapsed
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,NaN,NaN,facebook,0,en,direct,...,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN,NaN,NaN,NaN
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,38,facebook,0,en,seo,...,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN,NaN,NaN,NaN
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56,basic,3,en,direct,...,untracked,Web,Windows Desktop,IE,US,NaN,NaN,NaN,NaN,NaN
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42,facebook,0,en,direct,...,untracked,Web,Mac Desktop,Firefox,other,NaN,NaN,NaN,NaN,NaN
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,NaN,41,basic,0,en,direct,...,untracked,Web,Mac Desktop,Chrome,US,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn import tree

In [12]:
df_2 = df.drop('user_id', axis = 1)
df_2.head()

,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,action,action_type,action_detail,device_type,secs_elapsed
0,2010-06-28,2009-03-19 04:32:55,NaT,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN,NaN,NaN,NaN
1,2011-05-25,2009-05-23 17:48:09,NaT,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN,NaN,NaN,NaN
2,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,NaN,NaN,NaN,NaN,NaN
3,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,NaN,NaN,NaN,NaN,NaN
4,2010-09-14,2009-12-08 06:11:05,2010-02-18,NaN,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,NaN,NaN,NaN,NaN,NaN


In [13]:
df_2['dac_year'] = df_2.date_account_created.dt.year
df_2['dac_month'] = df_2.date_account_created.dt.month
df_2['dac_day'] = df_2.date_account_created.dt.day

df_2['tfa_year'] = df_2.timestamp_first_active.dt.year
df_2['tfa_month'] = df_2.timestamp_first_active.dt.month
df_2['tfa_day'] = df_2.timestamp_first_active.dt.day

df_2['dfb_year'] = df_2.date_first_booking.dt.year
df_2['dfb_month'] = df_2.date_first_booking.dt.month
df_2['dfb_day'] = df_2.date_first_booking.dt.day

df_2 = df_2.drop(['date_account_created','timestamp_first_active','date_first_booking'],axis = 1)
df_2.head()


,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,...,secs_elapsed,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,dfb_year,dfb_month,dfb_day
0,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,...,NaN,2010,6,28,2009,3,19,NaN,NaN,NaN
1,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,...,NaN,2011,5,25,2009,5,23,NaN,NaN,NaN
2,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,...,NaN,2010,9,28,2009,6,9,2010,8,2
3,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,...,NaN,2011,12,5,2009,10,31,2012,9,8
4,NaN,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,...,NaN,2010,9,14,2009,12,8,2010,2,18


In [14]:
#df_2 = df_2[(df_2.age > 17 and df_2.age < )]
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

variables = ['gender','signup_method','language','country_destination','affiliate_channel',
             'affiliate_provider','first_affiliate_tracked','affiliate_provider',
             'first_affiliate_tracked','signup_app','first_device_type','first_browser','action',
            'action_type','action_detail','device_type']

for item in variables:
    print item
    le.fit(df_2[item])
    encoded = le.transform(df_2[item])
    df_2[item] = encoded
    

gender
signup_method
language
country_destination
affiliate_channel
affiliate_provider
first_affiliate_tracked
affiliate_provider
first_affiliate_tracked
signup_app
first_device_type
first_browser
action
action_type
action_detail
device_type


In [15]:
df_2 = df_2.fillna(-1)
df_2.head()

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,...,secs_elapsed,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,dfb_year,dfb_month,dfb_day
0,0,-1,1,0,5,2,4,7,2,3,...,-1,2010,6,28,2009,3,19,-1,-1,-1
1,2,38,1,0,5,7,8,7,2,3,...,-1,2011,5,25,2009,5,23,-1,-1,-1
2,1,56,0,3,5,2,4,7,2,6,...,-1,2010,9,28,2009,6,9,2010,8,2
3,1,42,1,0,5,2,4,7,2,3,...,-1,2011,12,5,2009,10,31,2012,9,8
4,0,41,0,0,5,2,4,7,2,3,...,-1,2010,9,14,2009,12,8,2010,2,18


In [16]:
scaler = preprocessing.MinMaxScaler()
df_3 = df_2
for item in list(df_2.columns):
    df_3[item] = scaler.fit_transform(df_2[item])
    

df_3.head()

/Users/mirunapopa/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/mirunapopa/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/mirunapopa/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/mi

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,...,secs_elapsed,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,dfb_year,dfb_month,dfb_day
0,0.000000,0.000000,0.5,0.00,0.208333,0.285714,0.235294,1,0.666667,0.375,...,0,0.00,0.454545,0.900000,0,0.181818,0.600000,0.000000,0.000000,0.00000
1,0.666667,0.019355,0.5,0.00,0.208333,1.000000,0.470588,1,0.666667,0.375,...,0,0.25,0.363636,0.800000,0,0.363636,0.733333,0.000000,0.000000,0.00000
2,0.333333,0.028288,0.0,0.12,0.208333,0.285714,0.235294,1,0.666667,0.750,...,0,0.00,0.727273,0.900000,0,0.454545,0.266667,0.997520,0.692308,0.09375
3,0.333333,0.021340,0.5,0.00,0.208333,0.285714,0.235294,1,0.666667,0.375,...,0,0.25,1.000000,0.133333,0,0.818182,1.000000,0.998512,0.769231,0.28125
4,0.000000,0.020844,0.0,0.00,0.208333,0.285714,0.235294,1,0.666667,0.375,...,0,0.00,0.727273,0.433333,0,1.000000,0.233333,0.997520,0.230769,0.59375


In [25]:
from sklearn import preprocessing
labels = df_3['country_destination']
df_3 = df_3.drop('country_destination',axis = 1)

In [ ]:
X = df_3
y = labels
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
xgb.fit(X, y)
